In [ ]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
plots.style.use('fivethirtyeight')

In [ ]:
# http://inferentialthinking.com/notebooks/san_francisco_2015.csv
sf = Table.read_table('san_francisco_2015.csv').select(3, 11, 21)
sf.set_format(2, NumberFormatter(0))
sf = sf.where(2, are.above(10000))
sf.show(3)

In [ ]:
comp_bins = np.arange(0, 700000, 25000)
sf.hist(2, bins=comp_bins, unit='dollar')

In [ ]:
#suppose we only have access to a sample:

sample_from_population = sf.sample(200, with_replacement=False)  #unique workers 
sample_from_population.show(3)

In [ ]:
# what is the median compensation in the population? Presumably, you do not know it. 
np.median(sample_from_population.column(2))

#is this median accurate?

In [ ]:
sample_from_population.select("Total Compensation").hist(bins = np.arange(0, 700000, 25000))

## Sample variability

In [ ]:
# loop in order to see the variation of the medians. 

medians = []
repetitions = 100

for i in np.arange(repetitions):
    sample_from_population = sf.sample(200, with_replacement=False)
    medians.append(np.median(sample_from_population.column(2)))

Table().with_columns(
    'i', np.arange(100),
    'median', medians,
).scatter('i')


#what do you see on this scatter plot?

In [ ]:
#back to slides
#You have only ONE sample. 

## Bootstrap

In [ ]:
sample_from_population.show(3)

In [ ]:
np.median(sample_from_population.column(2))


In [ ]:
# it does the bootstrap for you

resample = sample_from_population.sample()
resample.show(3)


In [ ]:
resample.select("Total Compensation").hist(bins = np.arange(0, 700000, 25000))

In [ ]:
np.median(resample.column(2))

In [ ]:
#What is the difference between these two (original and sample) histograms?

resampled_medians = []

for i in np.arange(1000):
    resample = sample_from_population.sample()
    median = np.median(resample.column(2))
    resampled_medians.append(median)

Table().with_column(
    "Resampled median", resampled_medians
).hist(unit='dollar')

In [ ]:
comp_bins = np.arange(0, 700000, 25000)
sf.hist(2, bins=comp_bins, unit='dollar')

In [ ]:
# pull out the middle 95%
# need to use percentile method (will be covered in Lab 7)

print("95% of resampled medians were between", 
      percentile(2.5, resampled_medians),
      "and",
      percentile(97.5, resampled_medians))


In [ ]:
#In yellow: middle 95% interval
#Red Circle: True median


Table().with_column('Resampled median', resampled_medians).hist(0)

# Draw the line and dot
interval_95 = [percentile(2.5, resampled_medians),
               percentile(97.5, resampled_medians)]
plots.plot(interval_95, [0, 0], color='gold', lw=5)
pop_median = np.median(sf.column(2))
plots.scatter(pop_median, 0, color='red', s=400)

In [ ]:
# Doing it all over again, putting the cells together

sample_from_population = sf.sample(200, with_replacement=False)

resampled_medians = []

for i in np.arange(1000):
    resample = sample_from_population.sample()
    median = np.median(resample.column(2))
    resampled_medians.append(median)

Table().with_column('Resampled median', resampled_medians).hist(0)

# Draw the line and dot
interval_95 = [percentile(2.5, resampled_medians),
               percentile(97.5, resampled_medians)]
plt.plot(interval_95, [0, 0], color='gold', lw=5)
pop_median = np.median(sf.column(2))
plt.scatter(pop_median, 0, color='red', s=400)

## Intervals Estimates

In [ ]:
#sf = Table.read_table('http://inferentialthinking.com/notebooks/san_francisco_2015.csv').select(3, 11, 21)
#sf.set_format(2, NumberFormatter(0))
#sf = sf.where(2, are.above(10000))
#sf.show(3)

In [ ]:
# bootstrap as a method

def bootstrap_median(sample_from_population, label, repetitions):
    resampled_medians = []
    for i in np.arange(repetitions):
        resample = sample_from_population.sample()
        median = np.median(resample.column(label))
        resampled_medians.append(median)
    return resampled_medians

In [ ]:
# THE BIG SIMULATION: This one takes several minutes.

# Generate 100 intervals, in the table intervals

left_ends = make_array()
right_ends = make_array()

total_comps = sf.select(2)
for i in np.arange(100):
    sample_from_pop = total_comps.sample(200, with_replacement=False)
    medians = bootstrap_median(sample_from_pop, 'Total Compensation', 5000)
    left_ends = np.append(left_ends, percentile(2.5, medians))
    right_ends = np.append(right_ends, percentile(97.5, medians))
    print('Iteration:', i)

intervals = Table().with_columns(
    'Left', left_ends,
    'Right', right_ends
)

In [ ]:
intervals.show(3)


In [ ]:
(intervals
 .where('Left', are.below(pop_median))
 .where('Right', are.above(pop_median))
 .num_rows)

In [ ]:
#visual represenation 
replication_number = np.ndarray.astype(np.arange(1, 101), str)

intervals2 = Table(replication_number).with_rows(make_array(left_ends, right_ends))

plt.figure(figsize=(8,8))
n=100
for i in np.arange(n):
    ends = intervals2.column(i)
    plt.plot(ends, make_array(i+1, i+1), color='gold')
plt.plot(make_array(pop_median, pop_median), make_array(0, n), color='red', lw=2)
plt.xlabel('Median (dollars)')
plt.ylabel('Replication')
plt.title('Population Median and Intervals of Estimates');

In [ ]:
#What can we do with it? (slides)

## Baby Weights

In [ ]:
# http://inferentialthinking.com/notebooks/baby.csv
births = Table.read_table('baby.csv')
births.show(3)

In [ ]:
babies = births.select(0, 1)
babies.show(3)

In [ ]:
ratios = babies.with_column(
    'Ratio BW:GD', babies.column(0) / babies.column(1)
)
ratios.show(3)

In [ ]:
ratios.hist(2)

In [ ]:
np.median(ratios.column(2))

In [ ]:
resampled_medians = bootstrap_median(ratios, 2, 5000)
resampled_medians

In [ ]:
# 5000 bootstrap samples
# We've taken the median of each of these samples
# Plotted them on the histogram
# We've constructed the 95% confidence interval (in yellow)

interval_95 = make_array(
    percentile(2.5, resampled_medians),
    percentile(97.5, resampled_medians)
)

Table().with_column('Resampled median', resampled_medians).hist(0)
plt.plot(interval_95, [0, 0], color='gold', lw=8)
print('Approximate 95% Bootstrap Confidence Interval for the Population Median')
print(np.round(interval_95, 4))

In [ ]:
interval_80 = make_array(
    percentile(10, resampled_medians),
    percentile(90, resampled_medians)
)

Table().with_column('Resampled median', resampled_medians).hist(0)
plt.plot(interval_80, [0, 0], color='gold', lw=8)
print('Approximate 80% Bootstrap Confidence Interval for the Population Median')

print(np.round(interval_80, 4))

In [ ]:
#(back to slides)

In [ ]:
def bootstrap_mean(sample_from_population, label, repetitions):
    resampled_means = []
    for i in np.arange(repetitions):
        resample = sample_from_population.sample()
        mean = np.mean(resample.column(label))
        resampled_means.append(mean)
    return resampled_means

In [ ]:
def bootstrap_ci_mean(sample_from_population, label, repetitions):
    resampled_means = bootstrap_mean(sample_from_population, label, repetitions)
    
    interval_95 = make_array(
        percentile(2.5, resampled_means),
        percentile(97.5, resampled_means)
    )
    
    Table().with_column('Resampled mean', resampled_means).hist(0)
    plt.plot(interval_95, [0, 0], color='gold', lw=8)
    print('Approximate 95% Bootstrap Confidence Interval for Population Mean:')
    print(np.round(interval_95, 3))

In [ ]:
births.show(3)

In [ ]:
bootstrap_ci_mean(births, 'Maternal Age', 5000)
# how do we interpret that? (iclicker question)

In [ ]:
births.hist('Maternal Age')